In [1]:
import re

with open("conda_requirements.txt", "w") as crf:
    crf.truncate(0)
    with open("pip_requirements.txt", "w") as prf:
        prf.truncate(0)
        with open('all_requirements.txt', 'r') as f:
            for line in f.readlines():
                pypi_check = re.search(r'(.+)=pypi.*', line)
                if pypi_check:
                    print(re.sub(r'=', '==', pypi_check.group(1)).strip(), file=prf)
                else:
                    print(line.strip(), file=crf)

In [2]:
!conda install -y --file conda_requirements.txt

Channels:
 - defaults
 - conda-forge
 - nvidia
 - pytorch
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\nicho\.conda\envs\agent_sizes

  added / updated specs:
    - anyio==4.4.0=pyhd8ed1ab_0
    - argon2-cffi-bindings==21.2.0=py38h91455d4_4
    - argon2-cffi==23.1.0=pyhd8ed1ab_0
    - arrow==1.3.0=pyhd8ed1ab_0
    - asttokens==2.4.1=pyhd8ed1ab_0
    - async-lru==2.0.4=pyhd8ed1ab_0
    - attrs==23.2.0=pyh71513ae_0
    - babel==2.14.0=pyhd8ed1ab_0
    - backcall==0.2.0=pyh9f0ad1d_0
    - beautifulsoup4==4.12.3=pyha770c72_0
    - blas==1.0=mkl
    - bleach==6.1.0=pyhd8ed1ab_0
    - brotli-python==1.1.0=py38hd3f51b4_1
    - ca-certificates==2024.7.4=h56e8100_0
    - cached-property==1.5.2=hd8ed1ab_1
    - cached_property==1.5.2=pyha770c72_1
    - certifi==2024.7.4=py38haa95532_0
    - cffi==1.16.0=py38h91455d4_0
    - charset-normalizer==3.3.2=pyhd8ed1ab_0
    - cloudpickle==2.2.1=py38haa95532_0
    - colorama==0.4.6=pyhd8ed

In [3]:
!pip install --quiet -r pip_requirements.txt

In [4]:
import sys
import itertools
import pdb
import random
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable
import torch.optim as optim

# setting device on GPU if available, else CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
import os
import sys

original_working_directory = os.getcwd()
original_working_directory

'C:\\git_repos\\2024_05_12_speaker_agent_sizes'

In [6]:
def count_parameters(module):
    parameter_count = 0
    for params in module.parameters():
        parameter_count += params.numel()
    return (parameter_count, len(str(parameter_count)))

In [7]:
# We need this a few times

from transformers import ResNetForImageClassification

resnet = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

params_resnet, _ = count_parameters(resnet)

del resnet

In [8]:
parameter_sizes = []

# 1. Natural language does not emerge ’naturally’ in multi-agent dialog

This paper uses symbollic inputs

In [9]:
os.chdir(original_working_directory)
os.chdir('lang-emerge')

# Based on what is in train.py!

from chatbots import Team
from dataloader import Dataloader
import options # sets the default hyperparameters etc.
from time import gmtime, strftime

# read the command line options
options = options.read()
#------------------------------------------------------------------------
# setup experiment and dataset
#------------------------------------------------------------------------
data = Dataloader(options)
numInst = data.getInstCount()

params = data.params
# append options from options to params
for key, value in options.items():
  params[key] = value

#------------------------------------------------------------------------
# build agents, and setup optmizer
#------------------------------------------------------------------------
team = Team(params)

speaker = team.qBot

params, order = count_parameters(speaker)
parameter_sizes.append({
    'title': 'Natural language does not emerge ’naturally’ in multi-agent dialog',
    'params': params,
    'order': order
})
params, order

Arguments:
	  hiddenSize : 50
	   embedSize : 20
	 imgFeatSize : 20
	   qOutVocab : 3
	   aOutVocab : 4
	     dataset : data/64_synthetic.json
	     rlScale : 100.0
	   numRounds : 2
	    remember : False
	 negFraction : 0.8
	   batchSize : 1000
	   numEpochs : 1000000
	learningRate : 0.001
	      useGPU : False
Answerer(
  (inNet): Embedding(7, 20)
  (outNet): Linear(in_features=50, out_features=4, bias=True)
  (imgNet): Embedding(12, 20)
  (rnn): LSTMCell(80, 50)
)
Questioner(
  (inNet): Embedding(16, 20)
  (outNet): Linear(in_features=50, out_features=3, bias=True)
  (rnn): LSTMCell(20, 50)
  (predictRNN): LSTMCell(20, 50)
  (predictNet): Linear(in_features=50, out_features=12, bias=True)
)


(29885, 5)

## 2. Emergence of Grounded Compositional Language in Multi-Agent Populations

This paper uses symbollic inputs

In [10]:
os.chdir(original_working_directory)
os.chdir('emergent-language/modules')

from agent import AgentModule
from configs import default_agent_config

speaker = AgentModule(default_agent_config)

params, order = count_parameters(speaker)
parameter_sizes.append({
    'title': 'Emergence of Grounded Compositional Language in Multi-Agent Populations',
    'params': params,
    'order': order
})
params, order

(1870105, 7)

## 3. Emergence of Communication in an Interactive World with Consistent Speakers

This speaker agent uses images as input and the CNN is initialised as part of the speaker agent.

In [11]:
os.chdir(original_working_directory)
os.chdir('emergence-communication-cco/agent')

import agent_type
from vec_agent import VecAgent

speaker = VecAgent(
    agent_type.AgentType.speaker,
    acting=True # this parameter does not affect the speaker agent size
)

params, order = count_parameters(speaker.model)
parameter_sizes.append({
    'title': 'Emergence of Communication in an Interactive World with Consistent Speakers',
    'params': params,
    'order': order
})
params, order

(45253, 5)

## 4. Compositional Obverter Communication Learning From Raw Visual Input

This speaker agent uses images as input and the CNN is initialised as part of the speaker agent.

In [12]:
os.chdir(original_working_directory)
os.chdir('obverter')

from model import ConvModel # Not just a CNN but an agent

speaker = ConvModel(
    5 # vocabulary size, taken from the paper
)

params, order = count_parameters(speaker)
parameter_sizes.append({
    'title': 'Compositional Obverter Communication Learning From Raw Visual Input',
    'params': params,
    'order': order
})
params, order

(64676, 5)

## 5. Emergence of Compositional Language with Deep Generational Transmission

We have to temporarily install parlai for this one, then uninstall it, then clean up after ourselves.

In [13]:
!pip install parlai --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jsonschema 4.23.0 requires attrs>=22.2.0, but you have attrs 20.2.0 which is incompatible.
referencing 0.35.1 requires attrs>=22.2.0, but you have attrs 20.2.0 which is incompatible.
transformers 4.42.4 requires huggingface-hub<1.0,>=0.23.2, but you have huggingface-hub 0.17.3 which is incompatible.


In [14]:
os.chdir(original_working_directory)
os.chdir('evolang')

from bots import Questioner

params, order = count_parameters(Questioner(
    { 
        'q_out_vocab': 64, # Max seen in run.py
        'a_out_vocab': 64, # Max seen in run.py
        'task_vocab': 6, # Defined in train.py as len(dataset.task_defn) which based on the datasets folder is typically 6
        'embed_size': 20, # default from options.py
        'hidden_size': 100, # default from options.py
        'props': {'shapes': [0, 1, 2, 3], 'styles': [0, 1, 2, 3], 'colors': [0, 1, 2, 3]} # From the json in the datasets folder
    }
))
parameter_sizes.append({
    'title': 'Emergence of Compositional Language with Deep Generational Transmission',
    'params': params,
    'order': order
})
params, order

(107956, 6)

Reinstall newer version of attrs so that everything else can run!

In [15]:
!pip uninstall -y parlai --quiet

In [16]:
os.chdir(original_working_directory)

In [17]:
!conda install -y --file conda_requirements.txt

Channels:
 - defaults
 - conda-forge
 - nvidia
 - pytorch
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



In [18]:
!pip install --quiet -r pip_requirements.txt

## 6. The Emergence of Compositional Languages for Numeric Concepts Through Iterated Learning in Neural Agents

In [19]:
# The paper says they use an LSTM speaker like Havrylov and Titov (2017) "Emergence of language with multi-agent game..."
# Havrylov and Titov say they use an LSTM with hidden size 512 and vocabulary 10000
# And also a LeNet for embedding images

class HavrylovNet(nn.Module):
    # Based on https://github.com/lychengrex/LeNet-5-Implementation-Using-Pytorch/blob/master/LeNet-5%20Implementation%20Using%20Pytorch.ipynb

    # network structure
    def __init__(self):
        super(HavrylovNet, self).__init__()
        self.rnn = nn.LSTM(
            10, # LSTM input size is LeNet output size, i.e. 10
            512
        )
        self.projector = nn.Linear(512, 10000)

    def forward(self, img, wrd):
        '''
        One forward pass through the network.
        
        Args:
            x: input
        '''
        return F.softmax(self.projector(self.rnn(x)))

class LeNet(nn.Module):

    # network structure
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        '''
        One forward pass through the network.
        
        Args:
            x: input
        '''
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        '''
        Get the number of features in a batch of tensors `x`.
        '''
        size = x.size()[1:]
        return np.prod(size)

params, order = count_parameters(torch.nn.Sequential(LeNet(), HavrylovNet()))
parameter_sizes.append({
    'title': 'The Emergence of Compositional Languages for Numeric Concepts Through Iterated Learning in Neural Agents',
    'params': params,
    'order': order
})
params, order

(6264858, 7)

## 7. Ease-of-Teaching and Language Structure from Emergent Communication

In [20]:
os.chdir(original_working_directory)
os.chdir('Ease-of-teaching-and-language-structure/code')

import argparse
# The initial arguments from parse are copied from parse.py
def parse():
    parser = argparse.ArgumentParser(description='Referential game settings')

    parser.add_argument('--gpu', type=int, default=0, help='which gpu if we use gpu')
    parser.add_argument('--fname', type=str, default='test', help='folder name to save results')
    parser.add_argument('--seed', type=int, default=0)
    parser.add_argument('--jupyter', action='store_true') 
    parser.add_argument('--slambda', type=float, default=0.1, help='speaker regularization hyperparameter')
    parser.add_argument('--rlambda', type=float, default=0.1, help='listener regularization hyperparameter')
    parser.add_argument('--receiverNum', type=int, default=1, help='number of listeners in the population')
    parser.add_argument('--topk', type=int, default=3, help='number of top messages when we probe language')
    parser.add_argument('--evaluateSize', type=int, default=1000, help='the batch size of test objects when not enumeration')
    args_dict = vars(parser.parse_args([])) # convert python object to dict
    return args_dict

args = parse()  # parsed argument from CLI
args['device'] = torch.device("cuda:" + str(args['gpu']) if torch.cuda.is_available() else "cpu")
if not os.path.exists(args['fname']):
    os.makedirs(args['fname'])

# dataset hyperparameters
args['numColors'] = 8 # based on the paper https://arxiv.org/pdf/1906.02403
args['numShapes'] = 4  # based on the paper https://arxiv.org/pdf/1906.02403
args['attrSize'] = args['numColors'] + args['numShapes']  # colors + shapes

# game settings
args['vocabSize'] = 8  # based on the paper https://arxiv.org/pdf/1906.02403
args['messageLen'] = 2  # based on the paper https://arxiv.org/pdf/1906.02403
args['distractNum'] = 5  # including targets

# training hyperparameters
args['batchSize'] = 100  # total train data = batchSize * numIters
args['sLearnRate'] = 0.001  
args['rLearnRate'] = 0.001  

args['trainIters'] = 300000 # training
args['resetNum'] = 50  
args['resetIter'] = args['trainIters'] // args['resetNum']  # life of a receiver: 6K
args['deterResetNums'] = 30
args['deterResetIter'] = 1000

# population of receivers training
args['population'] = False

# model hyperparameters
args['hiddenSize'] = 100  # based on the paper https://arxiv.org/pdf/1906.02403

from models import Sender

params, order = count_parameters(Sender(args))
parameter_sizes.append({
    'title': 'Ease-of-Teaching and Language Structure from Emergent Communication',
    'params': params,
    'order': order
})
params, order

(46108, 5)

## 8. Compositional Languages Emerge in a Neural Iterated Learning Model

In [21]:
os.chdir(original_working_directory)
os.chdir('Neural_Iterated_Learning/models')

if "model" in sys.modules:
    del sys.modules["model"] # we used model before from a different repository!
from model import SpeakingAgent

os.chdir('../utils')
import conf

params, order = count_parameters(SpeakingAgent())
parameter_sizes.append({
    'title': 'Compositional Languages Emerge in a Neural Iterated Learning Model',
    'params': params,
    'order': order
})
params, order

(73872, 5)

## 9. Compositionality and Generalization in Emergent Languages

In [22]:
"""
The paper says

  Each input i of the reconstruction game is comprised of iatt attributes,
  each with ival possible values. We let i_{att} range from 2 to 4 and i_{val}
  from 4 to 100. We represent each attribute as a i_{val} one-hot vector. An
  input i is given by the concatenation of its attributes. 

  ...

  Both agents are implemented as single-layer GRU cells (Cho et al., 2014)
  with hidden states of size 500.
  
  Sender encodes i in a message m of fixed length c_{len} as follows. First,
  a linear layer maps the input vector into the initial hidden state of
  Sender. Next, the message is generated symbol-by-symbol by sampling from a
  Categorical distribution over the vocabulary cvoc, parameterized by a linear
  mapping from Sender’s hidden state.

  In practice, we fix [vocabulary size to 100].
"""

class CompGenNet(nn.Module):

    # network structure
    def __init__(self):
        super(CompGenNet, self).__init__()
        self.fc1   = nn.Linear(400, 500)
        self.gru   = nn.GRU(100, 500)
        self.fc2   = nn.Linear(500, 100)

    def forward(self, x):
        '''
        One forward pass through the network.
        
        Args:
            x: input
        '''
        hidden = self.fc1(x)
        out, hidden = self.gru(x, hidden)
        return self.fc2(out)

params, order = count_parameters(CompGenNet())
parameter_sizes.append({
    'title': 'Compositionality and Generalization in Emergent Languages',
    'params': params,
    'order': order
})

params, order

(1153600, 7)

## 10. Co-evolution of language and agents in referential games

In [23]:
os.chdir(original_working_directory)
os.chdir('cultural-evolution-engine/model')

from ShapesModels import ShapesSender
from visual_module import CNN

cnn = CNN(
    # paper says "The linear layer which followed the convolutional layers had output dimensions of 512"
    n_out_features=512
)

sender = ShapesSender(vocab_size=5, output_len=5, sos_id=0)
params, order = count_parameters(torch.nn.Sequential(cnn, sender))
parameter_sizes.append({
    'title': 'Co-evolution of language and agents in referential games',
    'params': params,
    'order': order
})

params, order

(1696909, 7)

## 11. Inductive Bias and Language Expressivity in Emergent Communication

This repository requires us to install a package from a git repository.

In [24]:
!pip install git+https://github.com/facebookresearch/EGG.git --quiet

In [33]:
os.chdir(original_working_directory)
os.chdir('GameBias-EmeCom2020')

if "numpy" in sys.modules:
    del sys.modules["numpy"]

from modules import DspritesSenderCNN

params, order = count_parameters(DspritesSenderCNN())
parameter_sizes.append({
    'title': 'Inductive Bias and Language Expressivity in Emergent Communication',
    'params': params,
    'order': order
})

params, order

C:\Users\nicho\.conda\envs\agent_sizes\lib\site-packages\egg\core\language_analysis.py:11: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  import numpy as np


ImportError: DLL load failed while importing lapack_lite: The specified module could not be found.

## 12. Capacity, Bandwidth, and Compositionality in Emergent Language Learning

In [ ]:
import sys

os.chdir(original_working_directory)

# Read in the file
with open('cbc-emecom/main.py', 'r') as file:
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('parser.add_argument', '#')

# Write the file out again
with open('cbc-emecom/main.py', 'w') as file:
  file.write(filedata)
os.chdir("cbc-emecom")

# sys.argv = ["main.py", "--num-binary-messages", "24", "--num-digits", "6", "--embedding-size-sender", "40", "--project-size-sender", "60", "--num-lstm-sender", "300", "--num-lstm-receiver", "325", "--embedding-size-receiver", "125", "--save-str", "<SAVE_STR>"]

from main import CompCap

config = {
    'device': device,
    'num_binary_messages': 24,
    'seed': 0,
    
    # problem size
    'batch_size': 100,
    'num_digits': 6,
    'signature_size': 2,
    
    # network params
    'embedding_size_sender': 40,
    'project_size_sender': 60,
    'num_lstm_sender': 300,
    'num_lstm_receiver': 325,
    'embedding_size_receiver': 125,
    
    # optimization params
    'learning_rate': 3e-4,
    'weight_decay': 1e-4,
    'output_loss_penalty': 1,
    'weight_norm_penalty': 1e-4,
    'temp': 1,
    'max_iters': 200000,
    'train_acc': 0.60,
    'trainval_acc': 0.60,
    # logging/printing
    'trainval_interval': 50,
    'model_dir': None,
    'save_str': '',
    'log_dir': "./logs",
    'save_dir': "./models"
}

# parameter_count = 0

capacity_bandwidth_compositionality = CompCap(config)

parameter_count = 0
for name, module in capacity_bandwidth_compositionality.named_modules():
    if name.startswith('sender'):
        for params in module.parameters():
            parameter_count += params.numel()

params, order = (parameter_count, len(str(parameter_count)))
parameter_sizes.append({
    'title': 'Capacity, Bandwidth, and Compositionality in Emergent Language Learning',
    'params': params,
    'order': order
})

params, order

## 13. Emergent Communication at Scale

In [ ]:
from transformers import ResNetForImageClassification

resnet = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

params_resnet, _ = count_parameters(resnet)

del resnet

"""
The paper says:

The speaker’s network architecture is composed of several components to transform the
target image x into a message m = (wt)T −1
t=0 :
• The encoder f is a fixed Resnet-50 architecture that has been previously trained on Ima-
genet with the BYOL algorithm. The resulting embedding f (x) is of size 2048.
• The RNN hθ used is an LSTM of hidden size 256. Therefore the core state zt,θ is of size
512.
• The core-state adapter cθ is a linear layer with input size 2048 and an output size of 512 that
allows to transform the embedding f (x) into an appropriate core state z−1,θ = cθ (f (x)).
We split z−1,θ into two equal parts to obtain the initial hidden state zh,−1,θ and the initial
cell state zc,−1,θ .
• The word embedder gθ associates to each discrete symbols in W ∪ {sos} an embedding
of size 10.
• The value head vθ first selects the hidden part zh,t,θ of the core state zt,θ and then applies
a linear layer of output size 1.
• The policy head πθ first selects the hidden part zh,t,θ of the core state zt,θ and then applies
a linear layer of output size |W| to obtain logit
"""

class ScaleNet(nn.Module):

    # network structure
    def __init__(self):
        super(ScaleNet, self).__init__()
        self.core_state_adapter = nn.Linear(2048, 512)
        self.word_embedder = nn.Linear(20, 10)
        self.rnn = nn.LSTM(10, 256)
        self.value_head = nn.Linear(256, 10)
        self.policy_head = nn.Linear(256, 10)

    def forward(self, img, wrd):
        '''
        One forward pass through the network.
        
        Args:
            x: input
        '''
        x = self.core_state_adapter(x)
        return x

params, _ = count_parameters(ScaleNet())
params += params_resnet
order = len(str(params))
parameter_sizes.append({
    'title': 'Emergent Communication at Scale',
    'params': params,
    'order': order
})

params, order

## 14. Interaction history as a source of compositionality in emergent communication

In [ ]:
os.chdir(original_working_directory)
os.chdir('Interaction-history-as-a-source-of-compositionality/common')

from egg import core
from visual_classifier import Vision
os.chdir('../template_transfer')
from agents import Sender

cnn = Vision()
os.chdir('..')
speaker = core.RnnSenderReinforce(
    # parameters mostly from defaults in template_transfer/train.py
    agent=Sender(200, Vision.from_pretrained('vision_model.pth')),
    vocab_size=10, # from paper
    embed_dim=50,
    hidden_size=200,
    max_len=1,
    cell='rnn'
)

params, order = count_parameters(torch.nn.Sequential(cnn, speaker))
parameter_sizes.append({
    'title': 'Interaction history as a source of compositionality in emergent communication',
    'params': params,
    'order': order
})

params, order

## 15. Emergent communication of generalizations

In [ ]:
os.chdir(original_working_directory)
os.chdir('emergent-generalization/code/models')

from speaker import Speaker
from vision import ResNet18

vocab_size = 20 # vocab for the birds game as specified in the paper
embedding_size = 500 # from the paper

#This is based on what's in builder.py:
speaker = Speaker(ResNet18(), nn.Embedding(vocab_size + 3, embedding_size))

params, order = count_parameters(speaker)
parameter_sizes.append({
    'title': 'Emergent communication of generalizations',
    'params': params,
    'order': order
})

params, order

## 16. Compositionality Through Language Transmission, using Artificial Neural Networks

In [ ]:
os.chdir(original_working_directory)
os.chdir('neural-ilm/ilm')

from ilm_rnn_2018jan import AgentModel

speaker = AgentModel(
    # based on the defaults in ilm_rnn_2018jan.py
    5,
    10,
    50,
    4,
    20
)

params, order = count_parameters(speaker)
parameter_sizes.append({
    'title': 'Compositionality Through Language Transmission, using Artificial Neural Networks',
    'params': params,
    'order': order
})

params, order

## 17. TexRel: a Green Family of Datasets for Emergent Communications on Relations

In [ ]:
os.chdir(original_working_directory)
os.chdir('texrel/ref_task/models')

if "models" in sys.modules:
    del sys.modules["models"] # we used models before from a different repository!
 
from sender_model import SenderModel
from image_seq_embedders import PrototypicalSender
from decoders_differentiable import RNNDecoder
from pre_conv import StridedConv

speaker = SenderModel(
    # Arguments here based on defaults in ref_task/params_groups.py
    StridedConv(
        3, # Implied to be equal to grid_planes (see below) in ref_task/models/conv_models.py
        64, # Implied to be 64 by default in ref_task/models/conv_models.py
        0.2, # In ref_task/params_groups.py as --preconv-dropout
        4, # In ref_task/params_groups.py as --preconv-stride
        True # In ref_task/params_groups.py as --preconv-relu
    ),
    PrototypicalSender(
        128, # In ref_task/params_groups.py as --embedding-size
        0, # In ref_task/params_groups.py as --dropout
        (16, 16), # In ref_task/params_groups.py as --cnn-sizes, wants to be passed as an iterable here
        5 * 5, # In texrel/create_colletion.py as --grid-size, default 5, and needs to be multiplied by the size of the textures in the texrel images as shown in texrel/dataset_runtine.py, which (looking at the images in the paper) is about 5 pixels
        3, # In texrel/dataset_runtine.py as self.meta.grid_planes
        None, # In ref_task/params_groups.py as --cnn-max-pooling-size
        True # In ref_task/params_groups.py as --cnn-batch-norm
    ),
    RNNDecoder(
        128, # In ref_task/params_groups.py as --embedding-size
        10, # In ref_task/params_groups.py as --utt-len
        21 # In ref_task/params_groups.py as --vocab-size
    ),
    True
)

params, order = count_parameters(speaker)
parameter_sizes.append({
    'title': 'TexRel: a Green Family of Datasets for Emergent Communications on Relations',
    'params': params,
    'order': order
})

params, order

## 18. Disentangling Categorization in Multi-agent Emergent Communication

In [ ]:
os.chdir(original_working_directory)
os.chdir('disentangling_categorization/research_pool/config_archive/SEMIOSIS/111121-065556')

import json
with open('0_seed-0.json') as config: # This config uses a Resnet-50 as CNN
  state = json.loads(config.read())
state['device'] = 'cuda'

os.chdir(original_working_directory)
os.chdir('disentangling_categorization')

import os
os.environ["DATA_ROOT"] = "data"
os.environ["SAVE_ROOT"] = "SAVE_ROOT"
os.environ["DISENT_ROOT"] = "."

import model_builder

speaker = model_builder.build_complete_sender(state)

params, order = count_parameters(torch.nn.Sequential(speaker[0], speaker[1]))
parameter_sizes.append({
    'title': 'Disentangling Categorization in Multi-agent Emergent Communication',
    'params': params,
    'order': order
})

params, order

## 19. Emergence of hierarchical reference systems in multi-agent communication

In [ ]:
os.chdir(original_working_directory)
os.chdir('hierarchical_reference_game')

from archs import Sender

# based on what's in train.py
sender = Sender(256, 32, 4)

params, order = count_parameters(sender)
parameter_sizes.append({
    'title': 'Emergence of hierarchical reference systems in multi-agent communication',
    'params': params,
    'order': order
})

params, order

## 20. Emergent Communication: Generalization and Overfitting in Lewis Games

In [ ]:
os.chdir(original_working_directory)
os.chdir('Population/default_experiment/default_json')

import json
with open('agents.json') as config:
  sender_params = json.loads(config.read())['sender_default_reco']['sender_params']
#state['device'] = 'cuda'

os.chdir(original_working_directory)
os.chdir('Population/src/core')

# trainers.py indicates that the parameters of the sender should be considered to be
#   agent.sender.parameters() + agent.object_encoder.parameters()

from senders import build_sender

# sender_params = {
#     # based on 
#     "sender_type": "recurrent"
#     "sender_cell": "gru"
#     "sender_type": "recurrent"
# }
game_params = {
    "objects": {"object_type": "dummy"},
    "channel": {
        "voc_size": 10, # From default_json/onehotcompositionality_game.json
        "max_len": 10, # From default_json/onehotcompositionality_game.json
    }
}
sender = build_sender(sender_params, game_params)

params, order = count_parameters(sender)
parameter_sizes.append({
    'title': 'Emergent Communication: Generalization and Overfitting in Lewis Games',
    'params': params,
    'order': order
})

params, order

## 21. On the Correspondence between Compositionality and Imitation in Emergent Neural Communication

In [ ]:
'''
The paper (https://aclanthology.org/2023.findings-acl.787.pdf) 
says "The Sender is a single-layer GRU (Cho et al., 2014)
containing a fully-connected (FC) layer that maps the input
x to its first hidden state (dim=128)."
And also says
"Each input x denotes an object in an “attribute-
value world", where the object has n_{att} attributes,
and each attribute takes n_{val} possible values. We
represent x by a concatenation of n_{att} one-hot vec-
tors, each of dimension {n_val}.
...
We set n_{att} = 6, n_{val} = 10"
'''
class ImitNet(nn.Module):

    # network structure
    def __init__(self):
        super(ImitNet, self).__init__()
        self.fc = nn.Linear(6 * 10, 128)
        self.gru = nn.GRU(128, 128)

    def forward(self, x):
        '''
        One forward pass through the network.
        
        Args:
            x: input
        '''
       
        return self.gru(self.fc(x))

params, order = count_parameters(ImitNet())
parameter_sizes.append({
    'title': 'On the Correspondence between Compositionality and Imitation in Emergent Neural Communication',
    'params': params,
    'order': order
})

params, order

## 22. Compositionality with Variation Reliably Emerges in Neural Networks

In [ ]:
# The paper says
"""
The overall architecture used is intentionally similar to Chaabouni et al. (2020); Resnick et al. (2020) and
Guo et al. (2021) to allow comparison of results. The sender network is comprised of an embedding layer,
linear layer, and a GRU (Cho et al., 2014
"""

# Here is the same model we made to mock up Chaabouni (2020),
# with parameters changed according to configs/iclr_2023.jsonnet and the paper
class VariationNet(nn.Module):

    # network structure
    def __init__(self):
        super(VariationNet, self).__init__()
        self.vocab = 26 # 'signal_alphabet_size' from configs/iclr_2023.jsonnet
        self.embedding_size = 52 # 'embedding_size' from configs/iclr_2023.jsonnet
        self.input_size = 75 # sum of 'n_atoms_per_role' from configs/iclr_2023.jsonnet based on 1-hot vector input
        self.fc1   = nn.Linear(self.input_size, self.embedding_size)
        self.gru   = nn.GRU(self.vocab, self.embedding_size)
        self.fc2   = nn.Linear(self.embedding_size, self.vocab)

    def forward(self, x):
        '''
        One forward pass through the network.
        
        Args:
            x: input
        '''
        hidden = self.fc1(x)
        out, hidden = self.gru(x, hidden)
        return self.fc2(out)


params, order = count_parameters(VariationNet())
parameter_sizes.append({
    'title': 'Compositionality with Variation Reliably Emerges in Neural Networks',
    'params': params,
    'order': order
})

params, order

## 23. On the Word Boundaries of Emergent Languages Based on Harris's Articulation Scheme

In [ ]:
# The paper says
# "We have to define environments, agent architectures, and optimization methods for language emer-
# gence simulations. This paper adopts the framework of Chaabouni et al. (2020). ...
# We follow Chaabouni et al. (2020) as well for the architecture and optimization method."

# Same game and architecture as Chaabouni, so inherit param size results from that paper:
inherits_from = 'Compositionality and Generalization in Emergent Languages'
previous_calculation = [p for p in parameter_sizes if p['title'] == inherits_from][0]
params, order = previous_calculation['params'], previous_calculation['order']

parameter_sizes.append({
    'title': "On the Word Boundaries of Emergent Languages Based on Harris's Articulation Scheme",
    'params': params,
    'order': order
})
params, order

## 24. Lewis's Signaling Game as beta-VAE For Natural Word Lengths and Segments

In [ ]:
# Same game and architecture as Chaabouni, amnd "On the word boundaries"
# so inherit param size results from Chaabouni. Additional language model
# is conceptualised to be part of the receiver.

inherits_from = 'Compositionality and Generalization in Emergent Languages'
previous_calculation = [p for p in parameter_sizes if p['title'] == inherits_from][0]
params, order = previous_calculation['params'], previous_calculation['order']

parameter_sizes.append({
    'title': "Lewis's Signaling Game as beta-VAE For Natural Word Lengths and Segments",
    'params': params,
    'order': order
})

params, order

In [ ]:
import pandas as pd

os.chdir(original_working_directory)

pd.DataFrame(parameter_sizes).to_csv('output.csv', encoding='utf-8', index=False)